In [1]:
"""This script estimates TRFs for several models and saves them"""
from pathlib import Path
import re
import eelbrain
import mne


DATA_ROOT = Path("~").expanduser() / 'Data' / 'cocoha'

ENVELOPES_DIR = DATA_ROOT / 'envelopes'
EEG_DIR = DATA_ROOT / 'eeg'

TRF_DIR = DATA_ROOT / 'TRFs'
TRF_DIR.mkdir(exist_ok=True)

SUBJECTS = [path.name for path in EEG_DIR.iterdir() if re.match(r'S\d+', path.name)]

In [2]:
# Loop through subjects to estimate TRFs
for SUBJECT in SUBJECTS:
    print("-" * 50)
    print(f"Processing subject {SUBJECT}...")
    
    # Check if all TRF files already exist
    trf_dir = TRF_DIR / SUBJECT
    trf_attended_path = trf_dir / f"{SUBJECT}_attended_trf.pickle"
    trf_unattended_path = trf_dir / f"{SUBJECT}_unattended_trf.pickle"
    trf_padded_attended_path = trf_dir / f"{SUBJECT}_padded_attended_trf.pickle"
    trf_padded_unattended_path = trf_dir / f"{SUBJECT}_padded_unattended_trf.pickle"
    
    if all([trf_attended_path.exists(), trf_unattended_path.exists(), 
            trf_padded_attended_path.exists(), trf_padded_unattended_path.exists()]):
        print(f"All TRF files for {SUBJECT} already exist, skipping.")
        continue
    
    # Load EEG data from pickle files
    eeg_path = EEG_DIR / SUBJECT / f"{SUBJECT}_eeg.pickle"
    padded_eeg_path = EEG_DIR / SUBJECT / f"{SUBJECT}_padded_eeg.pickle"
    eeg = eelbrain.load.unpickle(eeg_path)
    padded_eeg = eelbrain.load.unpickle(padded_eeg_path)

    # Load envelope data (all 4 versions)
    attended_envelope_path = ENVELOPES_DIR / SUBJECT / f"{SUBJECT}_attended_envelope.pickle"
    unattended_envelope_path = ENVELOPES_DIR / SUBJECT / f"{SUBJECT}_unattended_envelope.pickle"
    padded_attended_envelope_path = ENVELOPES_DIR / SUBJECT / f"{SUBJECT}_padded_attended_envelope.pickle"
    padded_unattended_envelope_path = ENVELOPES_DIR / SUBJECT / f"{SUBJECT}_padded_unattended_envelope.pickle"
    
    attended_envelope = eelbrain.load.unpickle(attended_envelope_path)
    unattended_envelope = eelbrain.load.unpickle(unattended_envelope_path)
    padded_attended_envelope = eelbrain.load.unpickle(padded_attended_envelope_path)
    padded_unattended_envelope = eelbrain.load.unpickle(padded_unattended_envelope_path)

    print(f"EEG: {eeg}")
    print(f"Padded EEG: {padded_eeg}")
    print(f"Attended envelope: {attended_envelope}")
    print(f"Unattended envelope: {unattended_envelope}")
    print(f"Padded attended envelope: {padded_attended_envelope}")
    print(f"Padded unattended envelope: {padded_unattended_envelope}")

    # Estimate TRFs for attended and unattended conditions using boosting
    print("Estimating attended TRF...")
    trf_attended = eelbrain.boosting(
            eeg,
            attended_envelope,
            -0.100,
            1.000,
            error='l1',
            basis=0.050,
            partitions=5,
            test=1,
            selective_stopping=True
        )
    
    print("Estimating unattended TRF...")
    trf_unattended = eelbrain.boosting(
            eeg,
            unattended_envelope,
            -0.100,
            1.000,
            error='l1',
            basis=0.050,
            partitions=5,
            test=1,
            selective_stopping=True
        )
    
    print("Estimating padded attended TRF...")
    trf_padded_attended = eelbrain.boosting(
            padded_eeg,
            padded_attended_envelope,
            -0.100,
            1.000,
            error='l1',
            basis=0.050,
            partitions=5,
            test=1,
            selective_stopping=True
        )
    
    print("Estimating padded unattended TRF...")
    trf_padded_unattended = eelbrain.boosting(
            padded_eeg,
            padded_unattended_envelope,
            -0.100,
            1.000,
            error='l1',
            basis=0.050,
            partitions=5,
            test=1,
            selective_stopping=True
        )

    # Save all 4 TRFs
    trf_dir.mkdir(exist_ok=True, parents=True)
    
    eelbrain.save.pickle(trf_attended, trf_attended_path)
    eelbrain.save.pickle(trf_unattended, trf_unattended_path)
    eelbrain.save.pickle(trf_padded_attended, trf_padded_attended_path)
    eelbrain.save.pickle(trf_padded_unattended, trf_padded_unattended_path)

    print(f"Saved attended TRF to {trf_attended_path}")
    print(f"Saved unattended TRF to {trf_unattended_path}")
    print(f"Saved padded attended TRF to {trf_padded_attended_path}")
    print(f"Saved padded unattended TRF to {trf_padded_unattended_path}")

--------------------------------------------------
Processing subject S5...
EEG: <NDVar 'eeg': 192000 time, 64 sensor>
Padded EEG: <NDVar 'padded_eeg': 196200 time, 64 sensor>
Attended envelope: <NDVar 'attended': 192000 time>
Unattended envelope: <NDVar 'unattended': 192000 time>
Padded attended envelope: <NDVar 'padded_attended': 196200 time>
Padded unattended envelope: <NDVar 'padded_unattended': 196200 time>
Estimating attended TRF...
Estimating unattended TRF...
Estimating padded attended TRF...
Estimating padded unattended TRF...
Saved attended TRF to /Users/sylvestereley/Data/cocoha/TRFs/S5/S5_attended_trf.pickle
Saved unattended TRF to /Users/sylvestereley/Data/cocoha/TRFs/S5/S5_unattended_trf.pickle
Saved padded attended TRF to /Users/sylvestereley/Data/cocoha/TRFs/S5/S5_padded_attended_trf.pickle
Saved padded unattended TRF to /Users/sylvestereley/Data/cocoha/TRFs/S5/S5_padded_unattended_trf.pickle
--------------------------------------------------
Processing subject S2...
EE